# What Should I Play? - Steam Video Game Content Recommendation System
#### By: _Leticia Genao._

# Problem Statement
With many video games retailers in the market, the competition to retain customers is highly competitive with multiple platforms available for games to purchase and play through.  With various choices to make, keeping customers buying games from your platform is essential.  This project aims to develop a video game recommendation system for Steam users to input their favorite video game title and have similar games suggested in order to keep users from switching to other platforms such as Epic Games, OP.GG, Quixant, GOG, Humble Store, Itch.io, and Fanatical. Besides forming the recommender system, NLP will be used on the game description and used as feature in the system. This recommender system will be measured by cosine similarity scores.

This recommender system is based on a contextual strategy using the description, genre, game details, and tags. Null values were dropped from the dataframe.

- Data source: https://www.kaggle.com/datasets/trolukovich/steam-games-complete-dataset

# Background Info/Outside Research
According to an article by Stephen Totilo, author of Axios Gaming,  Steam leans on users’ [curation pages](https://store.steampowered.com/curators/topcurators/) and player reviews on the games themselves to raise awareness for games on their platform (Totilo, 2022). While users reviews and custom recommendation are an engaging way for users to submit their opinions and recommend titles to other players, what do you do when someone hasn't reviewed a game you want to push? That's where a content recommender system will come in handy. 

Additionally, Steam's [current](https://store.steampowered.com/recommender/76561199115116270) game recommendation system has limited capabilities. You can only search by tag, age, and popularity.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
steamdf=pd.read_csv('./data/steam_games.csv')

In [3]:
steamdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40833 entries, 0 to 40832
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   url                       40833 non-null  object 
 1   types                     40831 non-null  object 
 2   name                      40817 non-null  object 
 3   desc_snippet              27612 non-null  object 
 4   recent_reviews            2706 non-null   object 
 5   all_reviews               28470 non-null  object 
 6   release_date              37654 non-null  object 
 7   developer                 40490 non-null  object 
 8   publisher                 35733 non-null  object 
 9   popular_tags              37888 non-null  object 
 10  game_details              40313 non-null  object 
 11  languages                 40797 non-null  object 
 12  achievements              12194 non-null  float64
 13  genre                     40395 non-null  object 
 14  game_d

In [4]:
# Checking the only float value, which contains a lot of nulls
steamdf.describe()

,achievements
count,12194.000000
mean,77.237494
std,448.501848
min,1.000000
25%,12.000000
50%,21.000000
75%,38.000000
max,9821.000000


In [5]:
steamdf.shape

(40833, 20)

# Cleaning & Converting Date Column to datetime object

In [6]:
steamdf.head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,game_details,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price
0,https://store.steampowered.com/app/379720/DOOM/,app,DOOM,Now includes all three premium DLC packs (Unto...,"Very Positive,(554),- 89% of the 554 user revi...","Very Positive,(42,550),- 92% of the 42,550 use...","May 12, 2016",id Software,"Bethesda Softworks,Bethesda Softworks","FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...","English,French,Italian,German,Spanish - Spain,...",54.0,Action,"About This Game Developed by id software, the...",NaN,"Minimum:,OS:,Windows 7/8.1/10 (64-bit versions...","Recommended:,OS:,Windows 7/8.1/10 (64-bit vers...",$19.99,$14.99
1,https://store.steampowered.com/app/578080/PLAY...,app,PLAYERUNKNOWN'S BATTLEGROUNDS,PLAYERUNKNOWN'S BATTLEGROUNDS is a battle roya...,"Mixed,(6,214),- 49% of the 6,214 user reviews ...","Mixed,(836,608),- 49% of the 836,608 user revi...","Dec 21, 2017",PUBG Corporation,"PUBG Corporation,PUBG Corporation","Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","English,Korean,Simplified Chinese,French,Germa...",37.0,"Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$29.99,NaN
2,https://store.steampowered.com/app/637090/BATT...,app,BATTLETECH,Take command of your own mercenary outfit of '...,"Mixed,(166),- 54% of the 166 user reviews in t...","Mostly Positive,(7,030),- 71% of the 7,030 use...","Apr 24, 2018",Harebrained Schemes,"Paradox Interactive,Paradox Interactive","Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","English,French,German,Russian",128.0,"Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,NaN,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",$39.99,NaN
3,https://store.steampowered.com/app/221100/DayZ/,app,DayZ,The post-soviet country of Chernarus is struck...,"Mixed,(932),- 57% of the 932 user reviews in t...","Mixed,(167,115),- 61% of the 167,115 user revi...","Dec 13, 2018",Bohemia Interactive,"Bohemia Interactive,Bohemia Interactive","Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","English,French,Italian,German,Spanish - Spain,...",NaN,"Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,NaN,"Minimum:,OS:,Windows 7/8.1 64-bit,Processor:,I...","Recommended:,OS:,Windows 10 64-bit,Processor:,...",$44.99,NaN
4,https://store.steampowered.com/app/8500/EVE_On...,app,EVE Online,EVE Online is a community-driven spaceship MMO...,"Mixed,(287),- 54% of the 287 user reviews in t...","Mostly Positive,(11,481),- 74% of the 11,481 u...","May 6, 2003",CCP,"CCP,CCP","Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","English,German,Russian,French",NaN,"Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,NaN,"Minimum:,OS:,Windows 7,Processor:,Intel Dual C...","Recommended:,OS:,Windows 10,Processor:,Intel i...",Free,NaN


In [7]:
# Convert string dates to numeric datetime 
steamdf['release_date']=pd.to_datetime(steamdf['release_date'], errors='coerce')

In [8]:
# New column of just years to ease cleaning process
steamdf['date_year'] = steamdf['release_date'].dt.strftime('%Y')

In [9]:
# There are games set for future release - eliminate those so system only recommends games available now.
steamdf.sort_values(by=['release_date'], ascending = False).head()

,url,types,name,desc_snippet,recent_reviews,all_reviews,release_date,developer,publisher,popular_tags,...,languages,achievements,genre,game_description,mature_content,minimum_requirements,recommended_requirements,original_price,discount_price,date_year
38508,https://store.steampowered.com/app/918190/Zoro...,app,Zoroastra,Zoroastra the Game is an open world action-adv...,NaN,NaN,2025-04-03,Kyrus,"Kyrus,Kyrus","Early Access,Early Access,Action,Adventure,RPG...",...,English,NaN,"Action,Adventure,Indie,RPG,Early Access","About This Game Welcome to Zoroastra, A world...",Mature Content Description The developers de...,"Minimum:,Requires a 64-bit processor and opera...","Recommended:,Requires a 64-bit processor and o...",NaN,NaN,2025
5436,https://store.steampowered.com/app/955540/Tele...,app,Telecube Nightmare,A platformer with no jump or walking. The leve...,NaN,"1 user reviews,- Need more user reviews to gen...",2023-12-25,Magitech Games,"Magitech Games,Magitech Games","Action,Indie,Casual,Adventure",...,"English,Portuguese,Portuguese - Brazil",47.0,"Action,Adventure,Casual,Indie",About This Game Telecube Nightmare is a plat...,NaN,NaN,NaN,$2.99,$6.37,2023
31581,https://store.steampowered.com/app/942790/Magi...,app,Magical Star Pillars Anniversary Edition,NaN,NaN,NaN,2022-12-31,Toolkitz Games,NaN,"Action,Adventure,Indie,Casual",...,English,NaN,"Action,Adventure,Casual,Indie",About This Content This expansion includes al...,NaN,"Minimum:,OS:,Vista,Processor:,Intel Dual Core ...","Recommended:,OS:,Windows 10",Free,NaN,2022
31275,https://store.steampowered.com/app/850700/8in1...,app,8-in-1 IQ Scale Bundle - Quirky Jerk (OST),NaN,NaN,NaN,2022-07-25,ALEKSANDER CHEPAIKIN,NaN,Indie,...,"English,French,Italian,German,Spanish - Spain,...",NaN,Indie,About This Content The soundtrack of the game...,NaN,NaN,NaN,NaN,NaN,2022
35215,https://store.steampowered.com/app/923830/WAIF...,app,WAIFU WARS ONLINE,"Kill, loot, and level up with other players in...",NaN,NaN,2022-02-02,WAIFU Defense Force,"WAIFU Defense Force,WAIFU Defense Force","Action,RPG,Massively Multiplayer,Indie,Gore,Vi...",...,English,NaN,"Action,Indie,Massively Multiplayer,RPG","About This Game WAIFU , or W eaponized AI F ...",Mature Content Description The developers de...,NaN,NaN,NaN,NaN,2022


In [10]:
steamdf.isnull().sum()

url                             0
types                           2
name                           16
desc_snippet                13221
recent_reviews              38127
all_reviews                 12363
release_date                 4126
developer                     343
publisher                    5100
popular_tags                 2945
game_details                  520
languages                      36
achievements                28639
genre                         438
game_description             2913
mature_content              37936
minimum_requirements        19764
recommended_requirements    19758
original_price               5311
discount_price              26290
date_year                    4126
dtype: int64

In [11]:
# Remove nulls from date_year to change feature type
steamdf=steamdf.dropna(axis=0, subset=['date_year'])

In [12]:
steamdf.shape

(36707, 21)

In [13]:
steamdf['date_year']=steamdf['date_year'].astype(int)

In [14]:
steamdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36707 entries, 0 to 40832
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   url                       36707 non-null  object        
 1   types                     36707 non-null  object        
 2   name                      36707 non-null  object        
 3   desc_snippet              23703 non-null  object        
 4   recent_reviews            2690 non-null   object        
 5   all_reviews               28337 non-null  object        
 6   release_date              36707 non-null  datetime64[ns]
 7   developer                 36432 non-null  object        
 8   publisher                 31772 non-null  object        
 9   popular_tags              36594 non-null  object        
 10  game_details              36252 non-null  object        
 11  languages                 36704 non-null  object        
 12  achievements      

# General Cleaning

In [15]:
# Drop any duplicates
steamdf.drop_duplicates(inplace=True)

In [16]:
steamdf.shape

(36707, 21)

In [17]:
# Check type catagory
steamdf['types'].unique()

array(['app'], dtype=object)

In [18]:
steamdf.isnull().sum()

url                             0
types                           0
name                            0
desc_snippet                13004
recent_reviews              34017
all_reviews                  8370
release_date                    0
developer                     275
publisher                    4935
popular_tags                  113
game_details                  455
languages                       3
achievements                24528
genre                         373
game_description               66
mature_content              34033
minimum_requirements        16316
recommended_requirements    16310
original_price               1911
discount_price              25005
date_year                       0
dtype: int64

Majority of marture_content ratings are nulls.
- This project will run nlp on game_description. The system will use genre, popular_tags, and game_details as the features. The game recommendations will be categorized into price buckets using original_price.
- The rest of the columns will be dropped.

In [19]:
steamdf=steamdf.drop(columns=['url', 'all_reviews','languages', 'desc_snippet','recent_reviews','recommended_requirements','minimum_requirements', 'achievements',  'types', 'publisher', 'discount_price', 'mature_content'])

In [20]:
steamdf.isnull().sum()

name                   0
release_date           0
developer            275
popular_tags         113
game_details         455
genre                373
game_description      66
original_price      1911
date_year              0
dtype: int64

In [21]:
# Drop nulls.
steamdf=steamdf.dropna()

In [22]:
steamdf.isnull().sum()

name                0
release_date        0
developer           0
popular_tags        0
game_details        0
genre               0
game_description    0
original_price      0
date_year           0
dtype: int64

In [23]:
steamdf.shape

(33952, 9)

In [24]:
# Genre types needs to be seperated
steamdf['genre'].value_counts().tail()

Action,Adventure,RPG,Sports,Early Access                                                      1
Casual,Free to Play,Indie,Racing,Strategy                                                     1
Action,Adventure,Free to Play,Indie,Massively Multiplayer,Racing,RPG,Strategy,Early Access    1
Action,Adventure,Casual,Indie,Racing,RPG,Simulation,Strategy,Early Access                     1
Software Training                                                                             1
Name: genre, dtype: int64

In [25]:
steamdf['original_price'].unique().tolist()[:10] # Unique content of column in list form

['$19.99',
 '$29.99',
 '$39.99',
 '$44.99',
 'Free',
 '$59.99',
 '$14.99',
 '$49.99',
 'Free to Play',
 '1.020']

There's lots of variation in the price from values of whole prices to values of '1.020', and different types of strings. Let's take a closer look at the games.

In [26]:
# Only paid version is available on steam
steamdf.loc[(steamdf['original_price']=='Learn to Play Edition')]

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
1130,Sentinels of the Multiverse,2014-12-22,Handelabra Games Inc.,"Card Game,Superhero,Board Game,Strategy,Indie,...","Single-player,Multi-player,Online Multi-Player...","Indie,Strategy",About This Game Calling all Sentinels! Do yo...,Learn to Play Edition,2014


In [27]:
# Paid version and free demo available
steamdf.loc[(steamdf['original_price']=='Open Beta')]

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
36156,Darkness and a Crowd,2019-12-19,Seth Albertus,"Action,Adventure,RPG,Indie,Strategy,Experience...","Single-player,Steam Achievements,Captions avai...","Action,Adventure,Indie,RPG,Strategy","About This Game With help from a tiny friend,...",Open Beta,2019


In [28]:
# Paid version and free demo available
steamdf.loc[(steamdf['original_price']=='dungeon,builder,simulation,construct,build,simulator,TD,地城,地牢,地下城,經營,模擬,建造,2D,indie,獨立,模拟,经营,策略,独立,塔防,管理,開放世界')]

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
30344,Dungeon Builder S,2017-07-13,LIN KE HSI,"Simulation,Indie,Strategy,Base Building,Resour...",Single-player,"Indie,Simulation,Strategy",About This Game The adventurers are always cu...,"dungeon,builder,simulation,construct,build,sim...",2017


In [29]:
# Only paid version
steamdf.loc[(steamdf['original_price']=="650560")]

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
18639,Slime-san: Superslime Edition,2017-04-07,Fabraz,"Indie,Action,Adventure,Platformer,Great Soundt...","Single-player,Local Multi-Player,Local Co-op,S...","Action,Adventure,Indie",About This Game Synopsis Slime-san: Supersli...,650560,2017


In [30]:
# Only paid version - interesting, might have to filter out 
# first two results have games in english, but for last game english is not supported
steamdf.loc[(steamdf['original_price']=="试玩版")]

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
9292,Journey of Greed,2019-04-25,Dird Games,"Early Access,Strategy,Card Game,Indie,Board Ga...","Online Multi-Player,Cross-Platform Multiplayer...","Indie,Strategy,Early Access",About This Game The demo is out! Try it with ...,试玩版,2019
29607,Fantasy Sino-Japanese War 幻想甲午,2019-01-11,张八万工作室,"Early Access,Early Access,Indie,RPG,Strategy","Single-player,Partial Controller Support,Steam...","Indie,RPG,Strategy,Early Access",About This Game You unfortunately passed thro...,试玩版,2019
39355,风暴岛,2018-12-01,风暴工作室,"Early Access,Indie,Early Access,Casual,Strateg...","Online Multi-Player,Commentary available,Steam...","Adventure,Casual,Indie,Massively Multiplayer,R...",About This Game 国内研发团队，首款国产创新游戏作品。 本次仅为核心玩法试玩...,试玩版,2018


In [31]:
# Paid version and free english is not supported
steamdf.loc[(steamdf['original_price']=="灰烬天国 UnconqueredCastle")]

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
7483,不落城-Unconquered Castle,2018-10-12,灰烬天国,"Casual,Indie,Strategy,Visual Novel,Anime","Single-player,Profile Features Limited \r\n\t\...","Casual,Indie,Strategy",About This Game Visual Novel + SRPG. The sto...,灰烬天国 UnconqueredCastle,2018


In [32]:
# Original_price of 1.020 read in wrong. These games will be dropped.
steamdf=steamdf.loc[~(steamdf['original_price']=='1.020')]

## Check "Free" games

In [33]:
steamdf.loc[(steamdf['original_price']=="Free to Play") | (steamdf['original_price']=="Free To Play")].head()

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
12,TERA,2015-05-05,"Bluehole, Inc.","Free to Play,MMORPG,Massively Multiplayer,RPG,...","Multi-player,MMO,Co-op,Steam Trading Cards,Par...","Action,Adventure,Free to Play,Massively Multip...",About This Game TERA is at the forefront of ...,Free to Play,2015
18,War Robots,2018-04-05,Pixonic,"Free to Play,Robots,Action,Multiplayer,FPS,Mec...","Multi-player,Online Multi-Player,Steam Trading...","Action,Free to Play","About This Game War is raging, pilot! Are you...",Free To Play,2018
30,Tree of Savior (English Ver.),2016-03-28,"IMCGAMES Co.,Ltd.","MMORPG,Massively Multiplayer,Free to Play,RPG,...","MMO,Steam Trading Cards,In-App Purchases,Parti...","Free to Play,Massively Multiplayer,RPG","About This Game Tree of Savior (""TOS"") is an ...",Free to Play,2016
36,Wizard101,2008-09-02,KingsIsle Entertainment,"Massively Multiplayer,RPG,Magic,Card Game,Free...","Online Multi-Player,MMO,In-App Purchases,Profi...","Free to Play,Massively Multiplayer,RPG",About This Game Embrace your inner Wizard and...,Free To Play,2008
62,The Elder Scrolls®: Legends™,2017-05-31,"Sparkypants Studios, LLC","Free to Play,Card Game,Trading Card Game,Multi...","Single-player,Multi-player,Online Multi-Player...",Free to Play,About This Game PLAY ON YOUR OWN Story mode ...,Free to Play,2017


In [34]:
# Actual free full games - keep and replace with $0.00 - this is good to go and code
steamdf.loc[(steamdf['original_price']=="Free")].head()

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
4,EVE Online,2003-05-06,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,Free,2003
260,DCS World Steam Edition,2013-03-18,Eagle Dynamics SA,"Simulation,Flight,Free to Play,Military,Multip...","Single-player,Multi-player,Captions available,...","Free to Play,Simulation",About This Game Feel the excitement of flying...,Free,2013
738,Wasteland Survival,2019-06-03,"Joyloft Co., Limited","Free to Play,Survival,Multiplayer,Zombies,Adve...","Single-player,In-App Purchases,Steam is learni...","Action,Adventure,Free to Play,RPG,Simulation",About This Game Wasteland Survival - Post Ap...,Free,2019
801,Life is Strange - Episode 1,2015-01-29,"DONTNOD Entertainment,Feral Interactive (Mac),...","Story Rich,Choices Matter,Great Soundtrack,Fem...","Single-player,Steam Achievements,Full controll...","Action,Adventure",About This Game Episode 1 now FREE! Life is ...,Free,2015
1016,Eden Rising,2019-05-17,"Nvizzio Creations,Meridian4","Tower Defense,Adventure,Open World,Action,Stra...","Single-player,Multi-player,Online Multi-Player...","Action,Adventure,Indie,Strategy",About This Game Eden Rising is a brand new a...,Free,2019


In [35]:
# This game is free - replace with $0.00
steamdf.loc[(steamdf['original_price']=="Free Version")]

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
33129,Lords of Kingdoms,2018-04-11,ThankGame.com,"Free to Play,Strategy,Massively Multiplayer,RP...","Multi-player,Online Multi-Player,MMO,Online Co...","Free to Play,Indie,Massively Multiplayer,RPG,S...",About This Game Lords of Kingdoms is a multip...,Free Version,2018


In [36]:
# First two games are fully free. Third is a free trial.
steamdf.loc[(steamdf['original_price']=="Play for Free!")]

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
56,The Lord of the Rings Online™,2012-06-06,"Standing Stone Games, LLC","Free to Play,MMORPG,Massively Multiplayer,Fant...",MMO,"Free to Play,Massively Multiplayer,RPG",About This Game In The Lord of the Rings Onli...,Play for Free!,2012
85,Dungeons & Dragons Online®,2012-06-25,"Standing Stone Games, LLC","Free to Play,RPG,MMORPG,Massively Multiplayer,...",MMO,"Action,Adventure,Free to Play,Massively Multip...",About This Game Enter a world of danger and a...,Play for Free!,2012
778,HITMAN™,2016-03-11,"IO Interactive A/S,Feral Interactive (Linux),F...","Stealth,Action,Assassin,Third Person,Singlepla...","Single-player,Steam Achievements,Full controll...",Action,About This Game Experiment and have fun in t...,Play for Free!,2016


In [37]:
# Dropping Hitman game trial
steamdf=steamdf.drop(index=778)

In [38]:
steamdf['original_price']=steamdf['original_price'].replace("Free", '$0.00')

In [39]:
steamdf['original_price']=steamdf['original_price'].replace("Play for Free!", '$0.00')

In [40]:
# Only one fully free game in this category
steamdf['original_price']=steamdf['original_price'].replace("Free Version", '$0.00')

In [41]:
steamdf['original_price']=steamdf['original_price'].replace(['Free to Play', 'Free To Play'], '$0.00')

## Remove non-full games (demos, DLC, beta, trials), unreleased games, and set price limit.

In [42]:
# Upon looking further, only the demo versions of these games are listed. The actual full games are not listed in the dataframe so these demos will be dropped.
# 30 games
steamdf=steamdf.loc[~(steamdf['original_price']=="Demo")]

In [43]:
# Remove games not released yet.
steamdf=steamdf.loc[(steamdf['date_year'] <= 2022)]

In [44]:
# Remove gamest list as demos
steamdf=steamdf[~steamdf['game_details'].str.contains('Game demo')]

In [45]:
# All games labeled Downloadable Content are purely DLC and not full games and will be dropped.
steamdf=steamdf[~steamdf['game_details'].str.contains('Downloadable Content')]

In [46]:
steamdf[steamdf['original_price'].str.contains('Trial')]

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
160,Shadow of the Tomb Raider,2018-09-14,"Eidos-Montréal,Crystal Dynamics,Nixxes software","Adventure,Lara Croft,Action,Female Protagonist...","Single-player,Steam Achievements,Full controll...","Action,Adventure",About This Game Experience Lara Croft’s defi...,Shadow of the Tomb Raider Free Trial,2018
2092,The Crew™,2014-12-01,Ivory Tower in collaboration with Ubisoft Refl...,"Racing,Open World,Multiplayer,Driving,Co-op,Ma...","Single-player,Multi-player,MMO,In-App Purchase...","Action,Massively Multiplayer,Racing",About This Game Your car is your avatar - fin...,The Crew Trial,2014
13261,Unholy Heights,2013-08-16,Petit Depotto,"Strategy,Tower Defense,Indie,Management,Simula...","Single-player,Steam Achievements,Steam Trading...","Indie,Simulation,Strategy","About This Game Welcome to Unholy Heights, a...",Unholy Heights Trial Version,2013


In [47]:
steamdf[steamdf['original_price'].str.contains('Beta')]

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
36156,Darkness and a Crowd,2019-12-19,Seth Albertus,"Action,Adventure,RPG,Indie,Strategy,Experience...","Single-player,Steam Achievements,Captions avai...","Action,Adventure,Indie,RPG,Strategy","About This Game With help from a tiny friend,...",Open Beta,2019


In [48]:
# Early Access are full available or free games.
steamdf[steamdf['popular_tags'].str.contains('Early Access')].head()

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year
1,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,$29.99,2017
3,DayZ,2018-12-13,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,$44.99,2018
8,They Are Billions,2017-12-12,Numantian Games,"Early Access,Base Building,Strategy,Zombies,Su...","Single-player,Steam Achievements,Steam Trading...","Strategy,Early Access",About This Game They Are Billions is a strat...,$29.99,2017
10,For The King,2018-04-19,IronOak Games,"RPG,Turn-Based Combat,Adventure,Online Co-Op,C...","Single-player,Multi-player,Online Multi-Player...","Adventure,Indie,RPG,Strategy","About This Game Into The Deep, a brand new a...",$19.99,2018
14,Stonehearth,2018-07-25,Radiant Entertainment,"City Builder,Building,Sandbox,Strategy,Surviva...","Single-player,Multi-player,Online Multi-Player...","Indie,Simulation,Strategy","About This Game In Stonehearth, you pioneer a...",$19.99,2018


In [49]:
# https://stackoverflow.com/questions/66096676/extracting-price-text-pandas
steamdf['clean_price']=steamdf['original_price'].str.extract(pat=r'\$(\d[\d.,]*)\b').astype(float)

In [50]:
# Price check
steamdf['clean_price'].sort_values(ascending = False).unique()[35:80]

array([100.  ,  99.99,  97.86,  96.28,  90.78,  89.99,  89.95,  89.9 ,
        88.94,  86.56,  85.9 ,  84.97,  84.93,  82.95,  82.93,  80.81,
        79.99,  79.95,  78.91,  76.95,  76.67,  75.93,  75.91,  74.99,
        71.93,  70.94,  70.84,  69.99,  69.95,  67.96,  67.9 ,  66.95,
        66.93,  65.93,  64.94,  64.89,  64.88,  64.87,  63.86,  62.58,
        61.68,  60.93,  60.91,  59.99,  59.98])

In [51]:
# Drop games above 99.99 as the prices tend to be mislabeld above this threshold.
steamdf=steamdf.loc[(steamdf['clean_price'] < 99.99)]

In [52]:
steamdf.head()

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year,clean_price
0,DOOM,2016-05-12,id Software,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","Single-player,Multi-player,Co-op,Steam Achieve...",Action,"About This Game Developed by id software, the...",$19.99,2016,19.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Multi-player,Online Multi-Player,Stats","Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,$29.99,2017,29.99
2,BATTLETECH,2018-04-24,Harebrained Schemes,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Single-player,Multi-player,Online Multi-Player...","Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,$39.99,2018,39.99
3,DayZ,2018-12-13,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Multi-player,Online Multi-Player,Steam Worksho...","Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,$44.99,2018,44.99
4,EVE Online,2003-05-06,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Multi-player,Online Multi-Player,MMO,Co-op,Onl...","Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,$0.00,2003,0.00


In [53]:
steamdf.shape

(20969, 10)

# Splitting Columns

In [54]:
# Put in list for recommender system
steamdf['developer']=steamdf['developer'].str.split(",")

## Clean game_details

In [55]:
steamdf['game_details']=steamdf['game_details'].str.split(",")

In [56]:
steamdf.head()

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year,clean_price
0,DOOM,2016-05-12,[id Software],"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","[Single-player, Multi-player, Co-op, Steam Ach...",Action,"About This Game Developed by id software, the...",$19.99,2016,19.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,[PUBG Corporation],"Survival,Shooter,Multiplayer,Battle Royale,PvP...","[Multi-player, Online Multi-Player, Stats]","Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,$29.99,2017,29.99
2,BATTLETECH,2018-04-24,[Harebrained Schemes],"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","[Single-player, Multi-player, Online Multi-Pla...","Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,$39.99,2018,39.99
3,DayZ,2018-12-13,[Bohemia Interactive],"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","[Multi-player, Online Multi-Player, Steam Work...","Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,$44.99,2018,44.99
4,EVE Online,2003-05-06,[CCP],"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","[Multi-player, Online Multi-Player, MMO, Co-op...","Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,$0.00,2003,0.00


In [57]:
steamdf['game_details'][4]

['Multi-player',
 'Online Multi-Player',
 'MMO',
 'Co-op',
 'Online Co-op',
 'Steam Trading Cards']

In [58]:
to_drop=['Commentary available','Full controller support', 'Includes Source SDK',  'Stats', 'Steam Achievements', 'Steam Workshop', 'In-App Purchases',
                                    'Steam Cloud','Steam Leaderboards', 'Steam Trading Cards','SteamVR Collectibles', 'Valve Anti-Cheat enabled','Steam Turn Notifications', 'Partial Controller Support', 'Mods (require HL2)', 'Mods',
                                    'Profile Features Limited\xa0\r\n\t\t\t\t\t\t\t\t\t', 'Steam is learning about this downloadable content\xa0\r\n\t\t\t\t\t\t\t\t\t',
                                    'Steam is learning about this game\xa0\r\n\t\t\t\t\t\t\t\t\t', 'Steam is learning about this software\xa0\r\n\t\t\t\t\t\t\t\t\t']

In [59]:
# https://stackoverflow.com/questions/61468401/remove-list-values-from-a-column-of-lists-based-on-another-list
# Iterate through and remove the to_drop categories.
steamdf=steamdf.assign(game_details=[[x for x in y if x not in to_drop] for y in steamdf.game_details])

In [60]:
steamdf['game_details'][4]

['Multi-player', 'Online Multi-Player', 'MMO', 'Co-op', 'Online Co-op']

In [61]:
# EDA purposes only
gd_result=steamdf['game_details'].str.join('|').str.get_dummies()

gd_result

,Captions available,Co-op,Cross-Platform Multiplayer,Includes level editor,Local Co-op,Local Multi-Player,MMO,Multi-player,Online Co-op,Online Multi-Player,Shared/Split Screen,Single-player
0,0,1,0,0,0,0,0,1,0,0,0,1
1,0,0,0,0,0,0,0,1,0,1,0,0
2,0,0,1,0,0,0,0,1,0,1,0,1
3,0,0,0,0,0,0,0,1,0,1,0,0
4,0,1,0,0,0,0,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
40811,0,1,0,0,1,1,0,1,1,1,1,1
40813,0,0,0,0,0,0,0,0,0,0,0,1
40815,0,0,0,0,0,0,0,0,0,0,0,1
40825,0,0,0,0,0,0,0,0,0,0,0,1


In [62]:
gd_count=gd_result.apply(gd_result.value_counts)

In [63]:
# Frequency of game details
gd_count.loc[(gd_count.index)].T.sort_values(1, ascending=False).head(25)

,0,1
Single-player,1194,19775
Multi-player,17743,3226
Online Multi-Player,18937,2032
Shared/Split Screen,19274,1695
Co-op,19538,1431
Local Multi-Player,19664,1305
Online Co-op,20060,909
Local Co-op,20119,850
Cross-Platform Multiplayer,20121,848
Includes level editor,20123,846


The most popular games in the dataset is the single-player category with 94% of the games labeled single, however these game details can overlap with games having one or more of these tags.

## Split genre

In [64]:
steamdf['genre']=steamdf['genre'].str.split(",")

In [65]:
steamdf.head()

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year,clean_price
0,DOOM,2016-05-12,[id Software],"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","[Single-player, Multi-player, Co-op]",[Action],"About This Game Developed by id software, the...",$19.99,2016,19.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,[PUBG Corporation],"Survival,Shooter,Multiplayer,Battle Royale,PvP...","[Multi-player, Online Multi-Player]","[Action, Adventure, Massively Multiplayer]",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,$29.99,2017,29.99
2,BATTLETECH,2018-04-24,[Harebrained Schemes],"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","[Single-player, Multi-player, Online Multi-Pla...","[Action, Adventure, Strategy]",About This Game From original BATTLETECH/Mec...,$39.99,2018,39.99
3,DayZ,2018-12-13,[Bohemia Interactive],"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","[Multi-player, Online Multi-Player]","[Action, Adventure, Massively Multiplayer]",About This Game The post-soviet country of Ch...,$44.99,2018,44.99
4,EVE Online,2003-05-06,[CCP],"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","[Multi-player, Online Multi-Player, MMO, Co-op...","[Action, Free to Play, Massively Multiplayer, ...",About This Game,$0.00,2003,0.00


In [66]:
genre_result=steamdf['genre'].str.join('|').str.get_dummies()

In [67]:
genre_result.shape

(20969, 24)

In [68]:
genre_result.columns.tolist()

['Accounting',
 'Action',
 'Adventure',
 'Animation & Modeling',
 'Audio Production',
 'Casual',
 'Design & Illustration',
 'Early Access',
 'Education',
 'Free to Play',
 'Game Development',
 'Indie',
 'Massively Multiplayer',
 'Movie',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing']

In [69]:
genre_count=genre_result.apply(genre_result.value_counts)

In [70]:
# Frequency of genres
genre_count.loc[(genre_count.index)].T.sort_values(1, ascending=False).head(25)

,0,1
Indie,6241,14728
Action,11702,9267
Casual,13139,7830
Adventure,13236,7733
Strategy,16855,4114
Simulation,16914,4055
RPG,17583,3386
Early Access,18642,2327
Free to Play,19673,1296
Sports,19954,1015


## Split popular_tags

In [71]:
steamdf['popular_tags']=steamdf['popular_tags'].str.split(",")

In [72]:
steamdf

,name,release_date,developer,popular_tags,game_details,genre,game_description,original_price,date_year,clean_price
0,DOOM,2016-05-12,[id Software],"[FPS, Gore, Action, Demons, Shooter, First-Per...","[Single-player, Multi-player, Co-op]",[Action],"About This Game Developed by id software, the...",$19.99,2016,19.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,2017-12-21,[PUBG Corporation],"[Survival, Shooter, Multiplayer, Battle Royale...","[Multi-player, Online Multi-Player]","[Action, Adventure, Massively Multiplayer]",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,$29.99,2017,29.99
2,BATTLETECH,2018-04-24,[Harebrained Schemes],"[Mechs, Strategy, Turn-Based, Turn-Based Tacti...","[Single-player, Multi-player, Online Multi-Pla...","[Action, Adventure, Strategy]",About This Game From original BATTLETECH/Mec...,$39.99,2018,39.99
3,DayZ,2018-12-13,[Bohemia Interactive],"[Survival, Zombies, Open World, Multiplayer, P...","[Multi-player, Online Multi-Player]","[Action, Adventure, Massively Multiplayer]",About This Game The post-soviet country of Ch...,$44.99,2018,44.99
4,EVE Online,2003-05-06,[CCP],"[Space, Massively Multiplayer, Sci-fi, Sandbox...","[Multi-player, Online Multi-Player, MMO, Co-op...","[Action, Free to Play, Massively Multiplayer, ...",About This Game,$0.00,2003,0.00
...,...,...,...,...,...,...,...,...,...,...
40811,Nanoui,2018-03-21,[Phoesion],"[Casual, RPG, Action, Adventure, Indie, Violen...","[Single-player, Multi-player, Online Multi-Pla...","[Action, Adventure, Casual, Indie, RPG]",About This Game The book of Nanoui has been ...,$6.99,2018,6.99
40813,Achievement Collector: Cat,2018-09-06,[QUIKIN Games],"[Action, Adventure]",[Single-player],"[Action, Adventure]",About This Game Achievement Collector: Cat in...,$0.99,2018,0.99
40815,SpaceBall in Cube,2018-08-26,[Enum],"[Action, Indie, Sports]",[Single-player],"[Action, Indie, Sports]",About This Game SpaceBall in Cube is a game f...,$9.99,2018,9.99
40825,Galactis,2018-08-15,[Gustavo Contreras],"[Action, Indie]",[Single-player],"[Action, Indie]",About This Game Revive the glory of bygone sp...,$0.99,2018,0.99


In [73]:
steamdf=steamdf.drop(columns=['original_price', 'release_date'])

In [74]:
steamdf.shape

(20969, 8)

In [75]:
steamdf.head()

,name,developer,popular_tags,game_details,genre,game_description,date_year,clean_price
0,DOOM,[id Software],"[FPS, Gore, Action, Demons, Shooter, First-Per...","[Single-player, Multi-player, Co-op]",[Action],"About This Game Developed by id software, the...",2016,19.99
1,PLAYERUNKNOWN'S BATTLEGROUNDS,[PUBG Corporation],"[Survival, Shooter, Multiplayer, Battle Royale...","[Multi-player, Online Multi-Player]","[Action, Adventure, Massively Multiplayer]",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,2017,29.99
2,BATTLETECH,[Harebrained Schemes],"[Mechs, Strategy, Turn-Based, Turn-Based Tacti...","[Single-player, Multi-player, Online Multi-Pla...","[Action, Adventure, Strategy]",About This Game From original BATTLETECH/Mec...,2018,39.99
3,DayZ,[Bohemia Interactive],"[Survival, Zombies, Open World, Multiplayer, P...","[Multi-player, Online Multi-Player]","[Action, Adventure, Massively Multiplayer]",About This Game The post-soviet country of Ch...,2018,44.99
4,EVE Online,[CCP],"[Space, Massively Multiplayer, Sci-fi, Sandbox...","[Multi-player, Online Multi-Player, MMO, Co-op...","[Action, Free to Play, Massively Multiplayer, ...",About This Game,2003,0.00


In [76]:
# EDA purposes

In [77]:
tag_result=steamdf['popular_tags'].str.join('|').str.get_dummies()

In [78]:
tag_result.shape

(20969, 376)

In [79]:
tag_result.columns.tolist()[:5]

['1980s', "1990's", '2.5D', '2D', '2D Fighter']

In [80]:
# TO REMOVE
# # This may not work in streamlit app - NOTE FOR LETI
# tag_result.columns = [col +"_tag" for col in tag_result.columns]

In [81]:
tag_result.head()

,1980s,1990's,2.5D,2D,2D Fighter,360 Video,3D,3D Platformer,3D Vision,4 Player Local,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
# https://sparkbyexamples.com/pandas/pandas-count-frequency-value-occurs-in-dataframe-column/
new_tag=tag_result.apply(tag_result.value_counts)

In [83]:
# Frequency of game tags
new_tag.loc[(new_tag.index)].T.sort_values(1, ascending=False).head(10)

,0,1
Indie,5558,15411
Action,10984,9985
Adventure,12190,8779
Casual,12332,8637
Singleplayer,15253,5716
Strategy,16350,4619
Simulation,16462,4507
RPG,17249,3720
Early Access,18073,2896
Great Soundtrack,18261,2708


Some of the most popular tags are indie, action, adventure, casual, singleplayer, simulation, and RPG.

In [84]:
# The top 150 tags only.
pd.set_option("display.max_rows", 150)
top_tags=new_tag.loc[(new_tag.index)].T.sort_values(1, ascending=False).head(150).T

In [85]:
top_tags.head()

,Indie,Action,Adventure,Casual,Singleplayer,Strategy,Simulation,RPG,Early Access,Great Soundtrack,...,City Builder,Dark Humor,Perma Death,Aliens,Level Editor,Flight,Match 3,Addictive,Abstract,Experimental
0,5558,10984,12190,12332,15253,16350,16462,17249,18073,18261,...,20780,20780,20782,20785,20789,20796,20796,20797,20798,20801
1,15411,9985,8779,8637,5716,4619,4507,3720,2896,2708,...,189,189,187,184,180,173,173,172,171,168


In [86]:
steamdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20969 entries, 0 to 40832
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              20969 non-null  object 
 1   developer         20969 non-null  object 
 2   popular_tags      20969 non-null  object 
 3   game_details      20969 non-null  object 
 4   genre             20969 non-null  object 
 5   game_description  20969 non-null  object 
 6   date_year         20969 non-null  int32  
 7   clean_price       20969 non-null  float64
dtypes: float64(1), int32(1), object(6)
memory usage: 2.0+ MB


In [87]:
# https://stackoverflow.com/questions/39512002/convert-whole-dataframe-from-lower-case-to-upper-case-with-pandas
# Lowercase the game names for recommender system
steamdf = steamdf.applymap(lambda s: s.lower() if type(s) == str else s)

In [88]:
steamdf.head()

,name,developer,popular_tags,game_details,genre,game_description,date_year,clean_price
0,doom,[id Software],"[FPS, Gore, Action, Demons, Shooter, First-Per...","[Single-player, Multi-player, Co-op]",[Action],"about this game developed by id software, the...",2016,19.99
1,playerunknown's battlegrounds,[PUBG Corporation],"[Survival, Shooter, Multiplayer, Battle Royale...","[Multi-player, Online Multi-Player]","[Action, Adventure, Massively Multiplayer]",about this game playerunknown's battleground...,2017,29.99
2,battletech,[Harebrained Schemes],"[Mechs, Strategy, Turn-Based, Turn-Based Tacti...","[Single-player, Multi-player, Online Multi-Pla...","[Action, Adventure, Strategy]",about this game from original battletech/mec...,2018,39.99
3,dayz,[Bohemia Interactive],"[Survival, Zombies, Open World, Multiplayer, P...","[Multi-player, Online Multi-Player]","[Action, Adventure, Massively Multiplayer]",about this game the post-soviet country of ch...,2018,44.99
4,eve online,[CCP],"[Space, Massively Multiplayer, Sci-fi, Sandbox...","[Multi-player, Online Multi-Player, MMO, Co-op...","[Action, Free to Play, Massively Multiplayer, ...",about this game,2003,0.00


In [89]:
steamdf.to_csv('./Data/clean_data.csv',index=False)

Proceed to notebook "2_steam_content_recommender" for nlp content recommender modeling.